In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
from tqdm import tqdm
#from google.colab import drive
import os
from matplotlib import style
style.use('fivethirtyeight')
from fuzzywuzzy import fuzz, process
import numpy as np
pd.options.mode.chained_assignment = None

C:\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [3]:
lq_original =  pd.read_csv('bcstores.csv')
print('The shape: %d x %d' % lq_original.shape)
print('There are %d unique values' % lq_original.name.shape[0])

The shape: 501 x 8
There are 501 unique values


In [4]:
beer_original = pd.read_csv('beeradvocate.csv')
print('The shape: %d x %d' % beer_original.shape)
print('There are %d unique values' % beer_original.name.shape[0])

The shape: 513 x 11
There are 513 unique values


In [5]:
lq = lq_original.copy()
beer = beer_original.copy()

In [6]:
beer['comb'] = beer.mnf.str.lower()+ ' '+ beer.name.str.lower()
beer.comb = beer.comb.str.lower().str.replace("brewing","").str.replace("brewery","")

# Drop duplicates: diff packing, diff type( can or bottle)

In [7]:
lq.name = lq.name.str.lower().str.replace(" tall can","").str.replace(" can","").str.replace(" 4-pack","").str.replace(" 6-pack","")

In [8]:
lq = lq.sort_values(['name','price']).drop_duplicates(subset = ['name'], keep = 'first')

# Drop variety pack

In [9]:
lq.drop(labels = lq[lq['style'].str.contains('Variety')].index, inplace = True)

In [10]:
lq.drop(labels = lq[lq['name'].str.contains('variety')].index, inplace = True)

In [11]:
lq.desc = lq.desc.str.replace("\\n","").replace("\\xa0","")

In [12]:
lq.drop(index = [8,173,179,193,194,199,213,235,246,261,333,426,366], inplace = True) #original index

In [13]:
#filter temp1 without row (has desc is "no info") to sort then drop
temp1 = lq[lq.desc != 'No info'].sort_values(['desc','price'], ascending = False).drop_duplicates(subset = ['desc'], keep = 'first')

In [14]:
#add lq 
lq = pd.concat([lq[lq.desc =='No info'],temp1])

In [15]:
lq.sort_index(inplace = True)

In [16]:
lq.shape

(371, 8)

# Match beer names from BCLiquorStore & Beeradvo

In [17]:
def ratio_per_word (word, string):
    ratio = process.extract(word, string, limit = 1, scorer = fuzz.token_set_ratio)

    #sort dictionary by values
    similarity = ratio[0][1] 
    beer_comb = ratio[0][0]
    return beer_comb, similarity        

In [18]:
# match all beer name from BCLiquorStore and Beeradvo
long_rate=[]
long_simi = []
for i in lq.name:
    beer_comb,similarity = ratio_per_word(i,beer.comb)
    long_rate.append(beer_comb)
    long_simi.append(similarity)

In [19]:
print(len(long_rate))
print(len(long_simi))

371
371


In [20]:
lq['beercomb'] = long_rate
lq['simi'] = long_simi
lq['confirm']= np.nan

In [21]:
lq_temp = lq.merge(beer[['name','alc','comb','mnf','style']],how='left',left_on='beercomb',right_on='comb').drop_duplicates('name_x')

In [22]:
lq_temp.shape

(371, 16)

In [23]:
lq_temp.loc[lq_temp.alc_y == 'no info','alc_y'] = 0

In [24]:
lq_temp.alc_y = lq_temp.alc_y.str.replace('[\[\'\]%]','').astype(float)

In [25]:
lq_temp.alc_x = lq_temp.alc_x.str.replace('%','').astype(float)

In [26]:
confirm_yes = lq_temp[(lq_temp.simi>80) & (lq_temp.alc_x == lq_temp.alc_y)].index

In [27]:
confirm_yes

Int64Index([  0,   1,   3,   4,   6,   7,   8,   9,  11,  15,
            ...
            364, 365, 367, 368, 369, 370, 376, 377, 379, 384],
           dtype='int64', length=167)

In [28]:
confirm_yes.shape

(167,)

In [29]:
#high accuracy
checking1 = lq_temp[~lq_temp.index.isin(confirm_yes) & (lq_temp.simi >= 80) & (np.abs(lq_temp.alc_x - lq_temp.alc_y) <= 1)]
checking1.shape

(58, 16)

In [30]:
checking1

,name_x,country,price,sku_link,alc_x,style_x,beer_type,desc,beercomb,simi,confirm,name_y,alc_y,comb,mnf,style_y
2,33 acres of ocean pale ale,Canada,$12.29,/product/837591,5.60,Pale Ale,Domestic Beer - BC Craft Beer,"A full flavoured beer integrated with a distinct floral hop gives a quality unique to our Pacific Northwest surroundings. This style is evolutionary from a IPA with an assertive hop, lower alcohol content, and slight pine aroma. Low in malt, this beer has a refined quality and a refreshing temperament.",33 acres co. 33 acres of ocean,92,NaN,33 acres of ocean,5.30,33 acres co. 33 acres of ocean,33 Acres Brewing Co.,Pale Ale - American
17,belgian moon - mango wheat,Canada,$11.49,/product/101624,5.40,Wheat,Domestic Beer - National Domestic Beer,"A cloudy yellow-orange appearance with a rich, white head. Aromas of mango and light honeyed-biscuits, with ripe mango flavours and a biscuity malt with hints of clove and honey. Finishes qucik and dry with some mango notes lingering.",old yale co. moon dance mango wheat,80,NaN,moon dance mango wheat,5.00,old yale co. moon dance mango wheat,Old Yale Brewing Co.,Wheat Beer - Witbier
31,cannery brewing - naramata nut brown ale 6s,Canada,$12.79,/product/772756,5.50,Brown Ale,Domestic Beer - BC Craft Beer,"Naramata Nut Brown is a traditional English-Style Brown Ale. British and Canadian malted barley give this full bodied beer a velvety smooth mouth feel. Traditional English hops balance the malt sweetness and provide a rich, complex flavour.",cannery company naramata,80,NaN,naramata,5.50,cannery company naramata,Cannery Brewing Company,Brown Ale - English
32,carib lager,Trinidad And Tobago,$11.99,/product/328419,5.20,Lager,Import Beer,No info,carib limited carib lager,100,NaN,Carib Lager,5.00,carib limited carib lager,Carib Brewery Limited,['Lager - Adjunct']
35,category 12 brewing - chromatic ipa,Canada,$13.29,/product/102035,7.00,India Pale Ale,Domestic Beer - BC Craft Beer,No info,category 12 wild ipa,86,NaN,wild ipa,6.80,category 12 wild ipa,Category 12 Brewing,IPA - Belgian
36,category 12 brewing - juicy data hazy ipa,Canada,$14.29,/product/234334,6.00,India Pale Ale,Domestic Beer - BC Craft Beer,"This naturally hazy, double dry-hopped IPA is bursting with tropical fruit, passionfruit, peach, grapefruit, and citrus flavours, and is brewed with four hop varieties. The palate is full-bodied with a creamy texture and noticeable bitterness on the finish. This is the perfect choice for any fan of hazy IPA’s.",category 12 juicy data hazy ipa,100,NaN,juicy data hazy ipa,6.10,category 12 juicy data hazy ipa,Category 12 Brewing,IPA - New England
45,corona - light,Mexico,$14.29,/product/617720,3.70,Lager,Import Beer,"Corona Light is a light bodied, dry, crisp and refreshing light beer. It is especially enjoyable with meals that call for simple, light refreshment. It is most appropriate for spicy, hot foods.",grupo modelo s.a. de c.v. corona light,100,NaN,Corona Light,4.10,grupo modelo s.a. de c.v. corona light,Grupo Modelo S.A. de C.V.,['Lager - Light']
48,corona light - sleek,Mexico,$26.79,/product/108657,3.40,Lager,Import Beer,No info,grupo modelo s.a. de c.v. corona light,80,NaN,Corona Light,4.10,grupo modelo s.a. de c.v. corona light,Grupo Modelo S.A. de C.V.,['Lager - Light']
50,czechvar,Czech Republic,$2.49,/product/591230,5.00,Lager,Import Beer,"Czechvar lager has a think, rich head and markedly golden yellow colour. The mild hop aroma balances well with the perfectly synchronized sweet-bitter taste. The body is medium to robust, which makes the drinker feel like taking another sip.","['budějovický budvar, n.p.'] czechvar dark cherry lager",100,NaN,Czechvar Dark Cherry Lager,4.40,"['budějovický budvar, n.p.'] czechvar dark cherry lager","['Budějovický Budvar, n.p.']",['Fruit and Field Beer']
51,czechvar lager,Czech Republic,$12.29,/product/445411,5.00,Lager,Import Beer,Czechvar pours a crystal clear golden colour. A light carbonation is evident as a few small bubbles rise to th

In [31]:
#incorrect beer names from checking1
confirm_no = [50,83,209,247,292,327,360,361]

#correct beer names after manually check checking1
confirm_yes = [*confirm_yes, *checking1[~checking1.index.isin(confirm_no)].index]

In [35]:
len(confirm_yes)

217

In [33]:
#checking 
checking2 = lq_temp[~lq_temp.index.isin(confirm_yes) & ~lq_temp.index.isin(confirm_no)]
checking2.shape

(146, 16)

In [40]:
#correct beer names after manually check checking2
confirm_yes = [*confirm_yes, *[34,38,47,62,70,93,111,117,123,146,148,228,230,243,274,282,306,310]]
#incorrect beer names after manually check checking2
confirm_no = [*confirm_no, *checking2[~checking2.index.isin([34,38,47,62,70,93,111,117,123,146,148,228,230,243,274,282,306,310])].index]

In [41]:
#assign labels to beer names
lq_temp.confirm[lq_temp.index.isin(confirm_yes)] = 'yes'
lq_temp.confirm[lq_temp.index.isin(confirm_no)] = 'no'

In [42]:
lq_temp[lq_temp.confirm.isnull()]

,name_x,country,price,sku_link,alc_x,style_x,beer_type,desc,beercomb,simi,confirm,name_y,alc_y,comb,mnf,style_y


In [43]:
lq_temp[lq_temp.confirm == 'yes'].shape

(235, 16)

In [44]:
lq_temp[lq_temp.confirm == 'no'].shape

(136, 16)